<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2025/blob/main/VLE_NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
eps =  np.finfo(np.float64).eps

#Bubble/Dew Point Calculations

In [3]:
p=Props(['Isopropanol','Water'])
R=8.314 #J/(mol K)

In [4]:
z=np.array([0.4,0.6])

In [5]:
def bubbleP(x, T):
    Pi = x*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

In [6]:
def dewP(y, T):
    Pisat = p.Pvap(T)
    P=1/np.sum(y/Pisat)
    return P, y*P/Pisat

In [7]:
xs= np.linspace(0,1,21)
T=350.
Ps = []
y1s = []
for x in np.c_[xs , 1-xs ]:
    P, y = bubbleP(x, T)
    Ps.append(P)
    y1s.append(y[0])
Ps = np.r_[Ps]
y1s=np.r_[y1s]

In [8]:
fig = make_subplots()
fig.add_scatter(x=xs,y=Ps, name='bubble', mode='lines')
fig.add_scatter(x=y1s,y=Ps, name='dew', mode='lines')
fig.update_layout(width=800,height=600, title=f'Benzene-Tolune VLE at T={T} K', xaxis_title='x, y', yaxis_title='Pressure (Pa)')

In [9]:
def T_estimate(P, x):
    return np.sum(x*1/(1/p.Tbn-R*np.log(P/101325)/p.HvapNB))

In [10]:
def bubbleT(x, P):
    T = sp.optimize.root_scalar(lambda T: bubbleP(x, T)[0] - P, x0=T_estimate(P,x), method='Newton').root

    return T, x*p.Pvap(T)/P

In [141]:
def dewT(y, P):
    T = sp.optimize.root_scalar(lambda T: dewP(y, T)[0] - P, x0=T_estimate(P,y), method='Newton').root

    return T, y*P/p.Pvap(T)


In [12]:
dewT(z, 101325)

(Array(367.28064221, dtype=float64),
 Array([0.25759495, 0.74240505], dtype=float64))

In [13]:
xs= np.linspace(0,1,21)
Ts=[]
P=101325
y1s = []
for x in np.c_[xs , 1-xs ]:
    T, y = bubbleT(x, P)
    Ts.append(T)
    y1s.append(y[0])
Ts = np.r_[Ts]
y1s=np.r_[y1s]
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=xs,y=Ts, name='bubble', mode='lines', row=1,col=1)
fig.add_scatter(x=y1s,y=Ts, name='dew', mode='lines', row=1,col=1)
fig.add_scatter(x=xs,y=y1s, mode='lines', row=1,col=2, name='y1')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', row=1,col=2, name='', line_color='grey')
fig.update_layout(width=800,height=400, title=f'Isopropanol-Water "Ideal" VLE at P={P} Pa', xaxis_title='x, y', yaxis_title='Temperature (K)')

In [14]:
Ts = np.linspace(300, 400, 11)

In [15]:
Pvaps = p.Pvap(Ts)
Pvaps

Array([[  6790.29724671,   3552.31744655],
       [ 12064.05957525,   6253.39565258],
       [ 20517.05378388,  10574.6348263 ],
       [ 33546.23763268,  17245.41582124],
       [ 52934.66372235,  27217.42951618],
       [ 80884.91899127,  41697.97094244],
       [120038.55644395,  62180.50843359],
       [173479.86160169,  90471.36101051],
       [244723.59609144, 128711.65030597],
       [337687.59196608, 179394.06652219],
       [456652.1078282 , 245374.36175369]], dtype=float64)

In [16]:
Pvaps[:,0]/Pvaps[:,1]

Array([1.91151195, 1.92920139, 1.94021393, 1.94522637, 1.94488108,
       1.93978069, 1.93048528, 1.91751135, 1.90133213, 1.88237882,
       1.86104247], dtype=float64)

In [17]:
alpha = 2.5
x1=np.linspace(0,1,21)
y1 = alpha*x1/(1-x1+alpha*x1)

In [18]:
fig = make_subplots(rows=1, cols=1)
fig.add_scatter(x=x1,y=y1, mode='lines')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', line_color='grey')
fig.update_layout(width=500,height=500,showlegend=False)

In [19]:
P = 101325. #Pa
T = (dewT(z, P)[0] + bubbleT(z, P)[0])/2

def rachford_rice(z, P, T):
    K = p.Pvap(T)/P
    def rr0(phi):
        return np.sum(z/(1+phi*(K-1))) - 1
    phi = sp.optimize.root_scalar(rr0, x0 = 0.5, method = 'newton').root
    x = z/(1+phi*(K-1))
    y = K*x
    return (phi, x, y)

In [20]:
rachford_rice(z, P, 1.05*(dewT(z, P)[0]))

(Array(-9.06050968e-17, dtype=float64),
 Array([0.4, 0.6], dtype=float64),
 Array([1.16165704, 0.92148751], dtype=float64))

#McCabe Thiele

In [21]:
alpha = 2.5
x1=np.linspace(0,1,21)
y1 = alpha*x1/(1-x1+alpha*x1)
#x1 = y1/(alpha + y1 - y1*alpha)

In [22]:
F = 1. #mol/s
z = 0.55 #mol fraction of more volatile component
q = 1.  #liquid fraction in feed, "quality"

xd = 0.95   #mole fraction of more volatile component in distillate
rec = 0.9  #recovery of more volatile component in distillate
D = rec*F*z/ xd  #distillate flow rate
B = F - D
xb = (1-rec)*F*z/B

R = 1.5 #reflux ratio
Vb = ((R+1)*D - (1-q)*F)/B  #boilup ratio in stripping section

x_op_intersect = (z/(1-q+eps) - xd/(R+1))/(R/(R+1) + q/(1-q+eps))

In [23]:
x_staircase = [xd]
y_staircase = [xd]
y=xd
for i in range(100):
    x = y/(alpha + y - y*alpha)
    x_staircase.append(x)
    y_staircase.append(y)
    if x<xb:
        break

    if x>=x_op_intersect:
        y =R/(R+1)*x + xd/(R+1)
    else:
        y= (Vb+1)/Vb*x - xb/Vb

    x_staircase.append(x)
    y_staircase.append(y)





In [24]:
fig = make_subplots(rows=1, cols=1)
fig.add_scatter(x=x1,y=y1, mode='lines')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', line_color='grey')
fig.add_scatter(x=[xd, 0], y=[xd, xd/(R+1)], mode='lines', line_color='grey')
fig.add_scatter(x=[z, 0], y=[z, z/(1-q + eps)], mode='lines', line_color='grey')
fig.add_scatter(x=[xb, (Vb+xb)/(Vb+1)], y=[xb, 1], mode='lines', line_color='grey')
fig.add_scatter(x=x_staircase, y=y_staircase, mode='lines', line_color='green')
fig.update_xaxes(range=[0, 1])
fig.update_yaxes(range=[0, 1])
fig.update_layout(width=500,height=500,showlegend=False)

#NRTL (non-ideal liquid VLE calculations)

In [25]:

x=np.array([0.6,0.4])
T=320.

In [26]:
p.NRTL_gamma(x, T)

Array([1.14838492, 2.00687694], dtype=float64)

In [27]:
#Tij = Aij + Bij/T + Cij * Ln(T) + Dij * T

In [28]:
tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
G=jnp.exp(-p.NRTL_alpha*tau)

In [29]:
xtauG = jnp.einsum('j,ji,ji->i',x,tau,G)
xG =  jnp.einsum('j,ji->i',x,G)
xtauGdivxG = xtauG/xG

In [30]:
xtauGdivxG

Array([0.57787343, 0.03730442], dtype=float64)

In [31]:
jnp.exp(xtauGdivxG + jnp.einsum('j,ij->i',x, G*(tau - xtauGdivxG[None,:])/xG[None,:]))

Array([1.14838492, 2.00687694], dtype=float64)

In [32]:
xG

Array([0.77120577, 0.98786277], dtype=float64)

In [33]:
def GexRT(x, T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G=jnp.exp(-p.NRTL_alpha*tau)
    xtauG = jnp.einsum('j,ji,ji->i',x,tau,G)
    xG =  jnp.einsum('j,ji->i',x,G)
    xtauGdivxG = xtauG/xG
    return jnp.sum(x*xtauGdivxG)

In [34]:
GexRT(x, T)

Array(0.36164582, dtype=float64)

In [35]:
jnp.exp(jax.grad(GexRT)(x, T))

Array([1.14838492, 2.00687694], dtype=float64)

In [36]:
def bubbleP_NRTL(x, T):
    g = p.NRTL_gamma(x, T)
    pi = x*g*p.Pvap(T)
    P = jnp.sum(pi)
    return P, pi/P

In [37]:
def bubbleT_NRTL(x, P):
    T = sp.optimize.root_scalar(lambda T: bubbleP_NRTL(x, T)[0] - P, x0=bubbleT(x,P)[0], method='Newton').root
    g = p.NRTL_gamma(x, T)
    return T, x*g*p.Pvap(T)/P

In [38]:
def dewP_NRTL(y, T):

    def eq(vec):
        x = vec[:-1]
        P = vec[-1]
        return np.r_[x*p.NRTL_gamma(x, T)*p.Pvap(T)/P - y , np.sum(x)-1.]

    P, x = dewP(y, T)  #assume ideal system for initial guess
    res=sp.optimize.root(eq, np.r_[x, P])
    x = res.x[:-1]
    P = res.x[-1]
    return P, x


In [39]:
def dewT_NRTL(y, P):

    def eq(vec):
        x = vec[:-1]
        T = vec[-1]
        return np.r_[x*p.NRTL_gamma(x, T)*p.Pvap(T)/P - y , np.sum(x)-1.]

    T, x = dewT(y, P)  #assume ideal system for initial guess
    res=sp.optimize.root(eq, np.r_[x, T])
    x = res.x[:-1]
    T = res.x[-1]
    return T, x

In [40]:
xs= np.linspace(0,1,101)
Ts=[]
P=101325.
y1s = []
for x in np.c_[xs , 1-xs ]:
    T, y = bubbleT_NRTL(x, P)
    Ts.append(T)
    y1s.append(y[0])
Ts = np.r_[Ts]
y1s=np.r_[y1s]
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=xs,y=Ts, name='bubble', mode='lines', row=1,col=1)
fig.add_scatter(x=y1s,y=Ts, name='dew', mode='lines', row=1,col=1)
fig.add_scatter(x=xs,y=y1s, mode='lines', row=1,col=2, name='y1')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', row=1,col=2, name='', line_color='grey')
fig.update_layout(width=800,height=400, title=f'Isopropanol-Water NRTL VLE at P={P} Pa', xaxis_title='x, y', yaxis_title='Temperature (K)')

In [114]:
def flashNRTL_PT(z, P, T):
    dewTres, dewxres=dewT_NRTL(z, P)
    bubbleTres, bubbleyres = bubbleT_NRTL(z, P)
    x0 = (T - dewTres)*(z - dewxres)/(bubbleTres - dewTres) + dewxres
    for i in range(100):
        # print(i, x0)
        K = p.NRTL_gamma(x0,T)*p.Pvap(T)/P

        def rr0(phi):
            return np.sum(z*(K-1)/(1+phi*(K-1)))

        # res = sp.optimize.root_scalar(rr0, bracket=(0,1))
        res = sp.optimize.root_scalar(rr0, x0=0.5, method='newton')
        # print(res)
        phi=res.root
        x = z/(1+phi*(K-1))
        if np.linalg.norm(x-x0)<1e-6:
            break
        x0 = x
    y = x*p.NRTL_gamma(x,T)*p.Pvap(T)/P
    return (phi, x, y)


In [115]:
F = 1. #1 mole/s of feed
z = np.array([0.5,0.5])
P = 101325. #Pa

dewTres, dewxres=dewT_NRTL(z, P)
bubbleTres, bubbleyres = bubbleT_NRTL(z, P)

#Set T for flash calculation to be midway between dew and bubble T, to ensure 2 phase present.
flashT = (dewTres+bubbleTres)/2

In [116]:
phires, xres, yres = flashNRTL_PT(z, P, flashT)

In [117]:
#feed enthalpy of subcooled liquid (10 K below bubble T)
feedT = bubbleT_NRTL(z,P)[0]-10.
feedH = F * p.Hl(z, feedT) #moles of feed * enthalpy per mole of feed

In [118]:
#moles of liquid (vapor) * enthalpy per mole of liquid (vapor)
liquidH = F*(1-phires)*p.Hl(xres, flashT)
vaporH = F*phires*p.Hv(yres,flashT)

In [119]:
#heating power (W) to maintain flash temperature
Q = vaporH + liquidH - feedH
Q

Array(38476.82319952, dtype=float64)

In [124]:
def flashNRTL_PQ(z, feedH, flashP, Q=0):

    def energybalance(flashT):
        phires, xres, yres = flashNRTL_PT(z, flashP, flashT)
        liquidH = F*(1-phires)*p.Hl(xres, flashT)
        vaporH = F*phires*p.Hv(yres,flashT)
        return liquidH + vaporH - feedH - Q

    res = sp.optimize.root_scalar(energybalance, bracket=(bubbleT_flashP,dewT_flashP), method='bisect')
    flashT = res.root
    phires, xres, yres = flashNRTL_PT(z, flashP, flashT)
    return (flashT, phires, xres, yres)




In [129]:
F = 1. #1 mole/s of feed
z = np.array([0.5,0.5])
flashP = 101325. #Pa
bubbleT_flashP = bubbleT_NRTL(z, flashP)[0]
dewT_flashP = dewT_NRTL(z, flashP)[0]

feedP = 2e5 #Pa
feedT = bubbleT_NRTL(z, feedP)[0]
#saturated liquid enthalpy at feedP
feedH = p.Hl(z, feedT)

In [130]:
feedT

Array(371.98474021, dtype=float64)

In [131]:
flashNRTL_PQ(z, feedH, flashP)

(353.73340480061927,
 Array(0.06419668, dtype=float64),
 Array([0.49356362, 0.50643638], dtype=float64),
 Array([0.59382434, 0.40617581], dtype=float64))

In [142]:
p=Props(['Acetone','Isopropanol','Water'])
F = 1. #1 mole/s of feed
z = np.array([0.2,0.2,0.6])
flashP = 101325. #Pa
bubbleT_flashP = bubbleT_NRTL(z, flashP)[0]
dewT_flashP = dewT_NRTL(z, flashP)[0]

feedP = 2e5 #Pa
feedT = bubbleT_NRTL(z, feedP)[0]
#saturated liquid enthalpy at feedP
feedH = p.Hl(z, feedT)

In [143]:
flashNRTL_PQ(z, feedH, flashP)

(341.38612034576124,
 Array(0.0618523, dtype=float64),
 Array([0.177503 , 0.1992297, 0.6232673], dtype=float64),
 Array([0.54122488, 0.21168371, 0.24709202], dtype=float64))